In [1]:
import pandas as pd
from ast import literal_eval
data = pd.read_csv('train.csv')



In [2]:
data['dic'] = data['problems'].apply(lambda x: literal_eval(x))
data['choice'] = data['dic'].apply(lambda x: x['choices'])
data['answer'] = data['dic'].apply(lambda x: x['answer'])
data['question'] = data['dic'].apply(lambda x: x['question'])

In [3]:
import random

def balance_choices(row):
    # `choices` 순서를 랜덤하게 섞음
    shuffled_choices = random.sample(row['choice'], len(row['choice']))
    
    # 새로운 `answer` 계산 (기존 answer의 choice 값이 새 choices에서의 위치)
    original_choice = row['choice'][row['answer']-1]
    new_answer = shuffled_choices.index(original_choice) + 1
    return {
        'choice': shuffled_choices,
        'answer': new_answer
    }

data[['choice', 'answer']] = data.apply(balance_choices, axis=1, result_type='expand')


In [4]:
data['answer'].value_counts()
# 분포는 5빼고 고루 분포됨. 5는 증강이 필요할듯

answer
4    489
3    465
1    444
2    420
5    213
Name: count, dtype: int64

In [5]:
print(data['choice'][0])
print(data['answer'][0])

['ㄱ, ㄷ', 'ㄱ, ㄴ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']
1


In [6]:
pd.read_csv('train.csv').iloc[0]['problems']
# choices와 answer이 잘 매칭되는 것을 볼 수 있음.


"{'question': '상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?', 'choices': ['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ'], 'answer': 2}"

In [7]:
import ast

# Apply 함수에서 올바른 Python 리터럴 형식 생성
data['problems'] = data.apply(
    lambda x: "{'question': '%s', 'choices': %s, 'answer': %d}" % (
        x['question'].replace("'", "\\'"),
        [choice.replace("'", "\\'") for choice in x['choice']],
        x['answer']
    ),
    axis=1
)
# literal_eval을 사용해 문자열을 Python 딕셔너리로 변환
data['problems_dict'] = data['problems'].apply(ast.literal_eval)


In [8]:
data[['id', 'paragraph', 'problems', 'question_plus']].to_csv('fixed_label_distribution_train.csv', index = False)